In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
import os
import csv


In [3]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())


[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:00<00:00, 9.30MB/s]


In [4]:

# 카테고리 클릭하기
from selenium.common.exceptions import NoSuchElementException
browser = webdriver.Chrome(service=service,options=chrome_options)
wait = WebDriverWait(browser, 10)
browser.maximize_window()
url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000200010006&fltDispCatNo=&prdSort=01&pageIdx=1&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat1000001000200010006_Small"
browser.get(url)
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#Contents')))

total_links = []

selector = f'#Contents > ul.cate_list_box > li:nth-child(3) > a'

next_tab = browser.find_element(By.CSS_SELECTOR, selector)
next_tab.click()

    # soup = BeautifulSoup(browser.page_source,'html.parser')
    # links = soup.find_all('a', class_=['prd_thumb'])
    # links = [_['href'] for _ in links]
    # total_links.append(links)
# beautiful 형식
while True:
    try:
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#Container > div.pageing > strong')))
        soup = BeautifulSoup(browser.page_source,'html.parser')
        links = soup.find_all('a', class_=['prd_thumb'])
        links = [_['href'] for _ in links]
        total_links.append(links)
        next_num = browser.find_element(By.CSS_SELECTOR, '#Container > div.pageing > strong + a')
        next_num.click()
    except Exception:
        print("페이지네이션 끝")
        break
browser.quit()

페이지네이션 끝


In [5]:
cnt=0
for i in total_links:
    for j in i:
        cnt+=1
cnt

60

In [6]:
total_links[1]

['https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000162680&dispCatNo=1000001000200010006&trackingCd=Cat1000001000200010006_Small',
 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000131581&dispCatNo=1000001000200010006&trackingCd=Cat1000001000200010006_Small',
 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000137963&dispCatNo=1000001000200010006&trackingCd=Cat1000001000200010006_Small',
 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000185226&dispCatNo=1000001000200010006&trackingCd=Cat1000001000200010006_Small',
 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000148558&dispCatNo=1000001000200010006&trackingCd=Cat1000001000200010006_Small',
 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000181230&dispCatNo=1000001000200010006&trackingCd=Cat1000001000200010006_Small',
 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A

In [43]:
from selenium.common.exceptions import NoSuchElementException
total_data = list()

for link in total_links:
    for product in link:
        browser = webdriver.Chrome(service=service,options=chrome_options)
        wait = WebDriverWait(browser, 10)
        browser.maximize_window()
        browser.get(product)

        # title, price
        title = browser.find_element(By.CSS_SELECTOR, '.prd_name').text
        time.sleep(2)
        price = browser.find_element(By.CSS_SELECTOR, '.price-2 > strong').text
        time.sleep(2)    
        # img
        # 컬러 상세 이미지 가져오기

        colors = browser.find_elements(By.CSS_SELECTOR, '.thumb-color')

        for color in range(len(colors)):
            data = list()
            browser.find_element(By.CSS_SELECTOR, f'#colrParet_{color}').click()
            color_img = browser.find_element(By.CSS_SELECTOR, f'#colrParet_{color} > span > img').get_attribute('src')
            time.sleep(2)
            main_img = browser.find_element(By.CSS_SELECTOR, '#mainImg').get_attribute('src')
            time.sleep(2)
            img_name = browser.find_element(By.CSS_SELECTOR, '#goodstxt').text
            

            # to data
            data.append(title)
            data.append(price)
            data.append(color_img)
            data.append(main_img)
            data.append(img_name)
            total_data.append(data)

        browser.quit()


In [49]:
len(total_data)

241

In [58]:
cheek_df = pd.DataFrame(total_data, columns=['title', 'price', 'color_img', 'main_img', 'color_name'])

In [63]:
cheek_df.to_csv("c:/final project/cheek_df.csv",sep=',',encoding="utf-8-sig")

In [71]:
cheek_df.title = cheek_df.title.str.replace('/','_')

In [101]:
# 이미지 저장할 폴더 생성
import os
titles = cheek_df.title.unique()
for title in titles:
    os.mkdir("c:/final project/images/"+str(title))


In [102]:
# 상품 이름 경로에 컬러명으로 이미지 저장

import urllib
import urllib.request

for item in range(len(cheek_df.index)):
    title = cheek_df.loc[item].title
    color_name = cheek_df.loc[item].color_name
    color_img = cheek_df.loc[item].color_img
    main_img = cheek_df.loc[item].main_img

    urllib.request.urlretrieve(main_img, "c:/final project/images/"+title+"/"+color_name+"(발색비교).jpg")
    urllib.request.urlretrieve(color_img, "c:/final project/images/"+title+"/"+color_name+"(상품색상).jpg")



In [117]:
cheek_df['이미지경로_발색비교']=cheek_df.apply(lambda x: x.품명 +"/"+ x.색상 + "(발색비교).jpg", axis=1)
cheek_df['이미지경로_상품색상']=cheek_df.apply(lambda x: x.품명 +"/"+ x.색상 + "(상품색상).jpg", axis=1)


In [120]:
cheek

,품목,품명,가격,색상,이미지경로_발색비교,이미지경로_상품색상
0,블러셔,[NEW] 롬앤 베러 댄 치크,"9,900",N02 바인 누드,[NEW] 롬앤 베러 댄 치크/N02 바인 누드(발색비교).jpg,[NEW] 롬앤 베러 댄 치크/N02 바인 누드(상품색상).jpg
1,블러셔,[NEW] 롬앤 베러 댄 치크,"9,900",N01 너티 누드,[NEW] 롬앤 베러 댄 치크/N01 너티 누드(발색비교).jpg,[NEW] 롬앤 베러 댄 치크/N01 너티 누드(상품색상).jpg
2,블러셔,[NEW] 롬앤 베러 댄 치크,"9,900",C01 피치칩,[NEW] 롬앤 베러 댄 치크/C01 피치칩(발색비교).jpg,[NEW] 롬앤 베러 댄 치크/C01 피치칩(상품색상).jpg
3,블러셔,[NEW] 롬앤 베러 댄 치크,"9,900",C02 블루베리칩,[NEW] 롬앤 베러 댄 치크/C02 블루베리칩(발색비교).jpg,[NEW] 롬앤 베러 댄 치크/C02 블루베리칩(상품색상).jpg
4,블러셔,[NEW] 롬앤 베러 댄 치크,"9,900",C03 피그칩,[NEW] 롬앤 베러 댄 치크/C03 피그칩(발색비교).jpg,[NEW] 롬앤 베러 댄 치크/C03 피그칩(상품색상).jpg
...,...,...,...,...,...,...
236,블러셔,[정한 PICK]바닐라코 프라이밍 베일 치크 6g _ 4colors,"11,600",PK01 글리머,[정한 PICK]바닐라코 프라이밍 베일 치크 6g _ 4colors/PK01 글리머...,[정한 PICK]바닐라코 프라이밍 베일 치크 6g _ 4colors/PK01 글리머...
237,블러셔,[정한 PICK]바닐라코 프라이밍 베일 치크 6g _ 4colors,"11,600",CR01 리파인드,[정한 PICK]바닐라코 프라이밍 베일 치크 6g _ 4colors/CR01 리파인...,[정한 PICK]바닐라코 프라이밍 베일 치크 6g _ 4colors/CR01 리파인...
238,블러셔,[정한 PICK]바닐라코 프라이밍 베일 치크 6g _ 4colors,"11,600",PK02 문리버,[정한 PICK]바닐라코 프라이밍 베일 치크 6g _ 4colors/PK02 문리버...,[정한 PICK]바닐라코 프라이밍 베일 치크 6g _ 4colors/PK02 문리버...
239,블러셔,"잉글롯 AMC 페이스 리퀴드 블러쉬 91, 93호","34,000",91,"잉글롯 AMC 페이스 리퀴드 블러쉬 91, 93호/91(발색비교).jpg","잉글롯 AMC 페이스 리퀴드 블러쉬 91, 93호/91(상품색상).jpg"


In [121]:
cheek.to_csv("c:/final project/cheek.csv",sep=',',encoding="utf-8-sig")